### Upload Packages

In [33]:
import sys
sys.path.append(r'C:\Users\ggugl\Desktop\Giulia\Projects\Track-Personal-Finance\src')

In [34]:
import pandas as pd
from utils.preprocessed import clean_df

In [35]:
import datetime
import xlrd
import re # to assign category
import numpy as np # required for PBI
import matplotlib # required for PBI
import os

### l) Upload data

In [38]:
path_root = "C:/Users/ggugl/Desktop/Giulia/Projects/Track-Personal-Finance/"
path_data = path_root + "data"
path_cc = path_data + "/raw/carta_di_credito/2022"
path_cd = path_data + "/raw/bancomat/2022"
path_b = path_data + "/raw/bonifici/2022"

In [39]:
#####_______ list of excel files _______#####
list_files_cc = os.listdir(path_cc)
list_files_b = os.listdir(path_b)
list_files_cd = os.listdir(path_cd)

In [ ]:
#####_______ Loop over list of files to append to empty dataframe  _______#####

"""Initialize empty file"""
df_list_1 = pd.DataFrame()

"""" Start Loop """

for f in list_files_cc:
    single_excel = path_cc + "/" + f
    data = pd.read_excel(single_excel)
    df_list_1 = df_list_1.append(data)

"""" End Loop """


# Negative values as "earnings"
df_list_1 = df_list_1[["Data operazione", "Descrizione operazione", "Importo in euro"]]
df_list_1["type"] = df_list_1["Importo in euro"].apply(lambda x: "earnings" if x < 0 else "expenditures")

"""Initialize empty file"""
df_list_2 = pd.DataFrame()

"""" Start Loop """

for f in list_files_b:
    single_excel = path_b + "/" + f
    data = pd.read_excel(single_excel)
    df_list_2 = df_list_2.append(data)

"""" End Loop """

# Positive values as "earnings"
df_list_2 = df_list_2.rename(columns={"Data contabile": "Data operazione", "Importo": "Importo in euro"})
df_list_2 = df_list_2[["Data operazione", "Descrizione operazione", "Importo in euro"]]
df_list_2["type"] = df_list_2["Importo in euro"].apply(lambda x: "earnings" if x > 0 else "expenditures")

"""Initialize empty file"""
df_list_3 = pd.DataFrame()

"""" Start Loop """
for f in list_files_cd:
    single_excel = path_cd + "/" + f
    data = pd.read_excel(single_excel)
    df_list_3 = df_list_3.append(data)

"""" End Loop """


# Negative values as "earnings"
df_list_3 = df_list_3.rename(columns={"Data contabile": "Data operazione", "Descrizione": "Descrizione operazione", "Importo": "Importo in euro"})
df_list_3 = df_list_3[["Data operazione", "Descrizione operazione", "Importo in euro"]]
df_list_3["type"] = df_list_3["Importo in euro"].apply(lambda x: "earnings" if x < 0 else "expenditures")


In [ ]:
# Append the above dataframes 
df_list = df_list_1.append(df_list_2)
df_list = df_list.append(df_list_3)

# clean tables and select interest columns

In [ ]:
# Delete Rows if they are not datetime 
df_list["Data operazione"] = pd.to_datetime(df_list["Data operazione"], errors="coerce").dt.date
df_list = df_list.dropna(subset=["Data operazione"])
df_list = clean_df(df_list)

In [ ]:
# Select interested columns
df_list = df_list[["data_operazione", "descrizione_operazione", "importo_in_euro", "type"]]
# Extract Year and Month from the transaction Data
df_list["year"] = pd.DatetimeIndex(df_list["data_operazione"]).year
df_list["month"] = pd.DatetimeIndex(df_list["data_operazione"]).month
# Remove digit as there is a column Amount
df_list["descrizione_operazione"] = df_list["descrizione_operazione"].str.replace('\d+|eur|,', '')
df_list = df_list.reset_index(drop=True)

### ll) Categories Column

In [47]:
# create a dictionary of mappings
dataset = df_list
mapping = {"ristorante|pizza|bistrot|geco|moku"                    : "pranzo o cena fuori", 
           "farmacia|farmacie"                                     : "farmacia",
           "supermercato|supermarket|supermercati|market|pescheria": "spesa supermercato",
           "italki|verbling"                                       : "lezioni di inglese",
           "amazon|amz"                                            : "amazon",
           "edreams|ryanair"                                       : "voli",
           "caffetteria|starbucks|bar"                             : "bar",
           "protiviti|iqera"                                       : "stipendio",
           "enjoy"                                                 : "carsharing",
           "netflix"                                               : "netflix",
           "prelievo|prelievi"                                     : "prelievo",
           "calzature|conbipel|nuna lie|tezenis|zuiki|decathlon"   : "shopping",
           "hotel"                                                 : "hotel",
           "telecomital"                                           : "ricarica telefono"}


In [48]:
#### _______________ Function to map the above list _______________ #### 
def map_column(x):
    for key in mapping:
        if re.findall(key, x):
            return mapping[key]
    return "altro"

In [ ]:
#### _______________ Map and fill the category column _______________ #### 
dataset["category"] = dataset["descrizione_operazione"].map(lambda x: map_column(x))
dataset["category"].value_counts()

### Write the final Dataframe obtained

In [50]:
dataset.to_excel("dataset_finance.xlsx", index = False)